In [1]:
import pandas as pd 
LAYER=32

In [2]:
languages = ['cpp','cs','java','js','php','python','sh','ts']

In [3]:
def get_label_df(lang):
    source_lang_path = f'data/ds67{lang}/output2/LFCLF_embedding_human_eval_deepseek-ai_deepseek-coder-6.7b-instruct_{LAYER}.parquet'
    label_lang_path = f'data/ds67{lang}/output2/LFCLF_embedding_human_eval_deepseek-ai_deepseek-coder-6.7b-instruct_1_label.parquet'
    # print(source_lang_path)
    # print(label_lang_path)
    df_source = pd.read_parquet(source_lang_path)
    df_label = pd.read_parquet(label_lang_path)
    # print(df_label.head(1))
    result = list()
    for idx,row in df_source.iterrows():
        row['label'] = 1 if df_label.at[idx, 'label'] else 0
        result.append(row)
    return pd.DataFrame(result)    

In [4]:
SETTING = 'cross_language'
out_dir = 'data/data_for_classify'
import os

for language in languages:
    test_df = get_label_df(language)
    trains = list()
    for otherlang in languages:
        if otherlang == language:
            continue
        trains.append(get_label_df(otherlang))
    train_df = pd.concat(trains)
    train_df = train_df.sample(frac = 1)
    out_dir_lang = os.path.join(out_dir,language)
    if not os.path.exists(out_dir_lang):
        os.mkdir(out_dir_lang)
    test_df.to_parquet(os.path.join(out_dir_lang,'test.parquet'))
    train_df.to_parquet(os.path.join(out_dir_lang,'train.parquet'))
    print(language,test_df.shape,train_df.shape)
    print(test_df['label'].value_counts())
    print(train_df['label'].value_counts())


cpp (1610, 11) (11186, 11)
label
0    846
1    764
Name: count, dtype: int64
label
1    5794
0    5392
Name: count, dtype: int64
cs (1577, 11) (11219, 11)
label
0    853
1    724
Name: count, dtype: int64
label
1    5834
0    5385
Name: count, dtype: int64
java (1579, 11) (11217, 11)
label
1    941
0    638
Name: count, dtype: int64
label
1    5617
0    5600
Name: count, dtype: int64
js (1610, 11) (11186, 11)
label
1    1046
0     564
Name: count, dtype: int64
label
0    5674
1    5512
Name: count, dtype: int64
php (1610, 11) (11186, 11)
label
1    817
0    793
Name: count, dtype: int64
label
1    5741
0    5445
Name: count, dtype: int64
python (1640, 11) (11156, 11)
label
1    1279
0     361
Name: count, dtype: int64
label
0    5877
1    5279
Name: count, dtype: int64
sh (1580, 11) (11216, 11)
label
0    1100
1     480
Name: count, dtype: int64
label
1    6078
0    5138
Name: count, dtype: int64
ts (1590, 11) (11206, 11)
label
0    1083
1     507
Name: count, dtype: int64
label
1    6

In [5]:
train_df.head(1)

,task_id,completion_id,num_tokens,generation,first_token_embedding,last_token_embedding,first_token_code_embedding,last_token_code_embedding,has_error,extracted_code,label
170,HumanEval_105_by_length,HumanEval_105_by_length_0,303,Here is the completed function:\n\n```php\n<?p...,"[-0.189453125, 0.33203125, -0.396484375, 0.104...","[0.2080078125, 0.451171875, 0.08642578125, 0.3...","[0.10107421875, 0.271484375, -0.11181640625, -...","[0.48046875, 0.3203125, -0.10888671875, 0.2128...",False,function by_length($arr) {\n $words = array...,0


In [6]:
SETTING = 'cross_language'
out_dir = 'data/data_for_classify'
import os
import re
train = list()
test = list()
dfs = list()
for language in languages:
    dfs.append(get_label_df(language))
data_df = pd.concat(dfs)

with open('data/train.txt') as f:
    train_id = [int(l.strip()) for l in f.readlines()]
with open('data/test.txt') as f:
    test_id = [int(l.strip()) for l in f.readlines()]

for _,row in data_df.iterrows():
    task_id = row['task_id']
    numbers = re.findall(r'\d+', task_id)
    if len(numbers) <= 0:
        train.append(row)
        continue
    if int(numbers[0]) in train_id:
        train.append(row)
    else:
        test.append(row)

train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

train_df = train_df.sample(frac = 1)
out_dir_lang = os.path.join(out_dir,'cross_task')
if not os.path.exists(out_dir_lang):
    os.mkdir(out_dir_lang)
test_df.to_parquet(os.path.join(out_dir_lang,'test.parquet'))
train_df.to_parquet(os.path.join(out_dir_lang,'train.parquet'))
print(language,test_df.shape,train_df.shape)
print(test_df['label'].value_counts())
print(train_df['label'].value_counts())


ts (2490, 11) (10306, 11)
label
1    1300
0    1190
Name: count, dtype: int64
label
1    5258
0    5048
Name: count, dtype: int64


In [7]:
train_df.head(10)

,task_id,completion_id,num_tokens,generation,first_token_embedding,last_token_embedding,first_token_code_embedding,last_token_code_embedding,has_error,extracted_code,label
596,HumanEval_10_make_palindrome,HumanEval_10_make_palindrome_6,187,"```javascript\n if (string === """") {\n ...","[0.1669921875, 0.25, 0.013427734375, 0.0664062...","[0.05322265625, 0.341796875, 0.0859375, 0.1757...","[0.09375, -0.051513671875, -0.265625, 0.294921...","[0.365234375, 0.169921875, 0.002166748046875, ...",False,"if (string === """") {\n return """";\n ...",0
188,HumanEval_25_factorize,HumanEval_25_factorize_8,218,Here is the completed JavaScript function:\n\n...,"[-0.1796875, 0.3203125, -0.361328125, 0.157226...","[0.25, 0.326171875, 0.109375, 0.318359375, -0....","[0.068359375, -0.0859375, 0.013916015625, -0.0...","[0.46484375, 0.21484375, -0.0888671875, 0.2265...",False,function factorize(n){\n let factors = [];\...,1
230,Python/23,Python/23_0,57,```python\ndef strlen(string: str) -> int:\n ...,"[0.087890625, 0.369140625, 0.08447265625, 0.03...","[0.12060546875, 0.39453125, 0.10302734375, 0.0...","[0.05908203125, 0.328125, -0.19140625, -0.0111...","[0.1640625, 0.1591796875, 0.041259765625, 0.10...",False,"def strlen(string: str) -> int:\n """""" Retur...",1
1154,HumanEval_136_largest_smallest_integers,HumanEval_136_largest_smallest_integers_4,152,```javascript\n let largestNegative = undefin...,"[0.1875, 0.2333984375, 0.033447265625, 0.05688...","[0.07958984375, 0.435546875, 0.2216796875, 0.1...","[0.1181640625, 0.015869140625, 0.06787109375, ...","[0.40625, 0.4296875, -0.2353515625, 0.12207031...",False,let largestNegative = undefined;\n let small...,0
686,HumanEval_90_next_smallest,HumanEval_90_next_smallest_6,291,Here is the completed function:\n\n```csharp\n...,"[-0.263671875, 0.30078125, -0.384765625, 0.150...","[0.158203125, 0.322265625, 0.0869140625, 0.308...","[0.053466796875, 0.171875, -0.2138671875, 0.20...","[0.3671875, 0.2490234375, -0.1591796875, 0.065...",False,if (lst.Count < 2) {\n return null...,0
874,HumanEval_73_smallest_change,HumanEval_73_smallest_change_4,194,Here is the completed function in JavaScript:\...,"[-0.216796875, 0.322265625, -0.384765625, 0.13...","[0.1875, 0.345703125, 0.050048828125, 0.227539...","[0.2373046875, 0.333984375, -0.1298828125, 0.0...","[0.353515625, 0.271484375, -0.1884765625, 0.08...",False,function smallest_change(arr){\n let change...,1
855,HumanEval_73_smallest_change,HumanEval_73_smallest_change_5,187,Here is the completed function in TypeScript:\...,"[-0.2099609375, 0.328125, -0.384765625, 0.1347...","[0.2333984375, 0.44921875, 0.050048828125, 0.3...","[0.248046875, 0.322265625, -0.2060546875, 0.00...","[0.38671875, 0.365234375, -0.19140625, 0.11865...",False,function smallest_change(arr: number[]): numbe...,1
697,HumanEval_119_match_parens,HumanEval_119_match_parens_7,275,Here is the completed function in JavaScript:\...,"[-0.1845703125, 0.322265625, -0.31640625, 0.13...","[0.193359375, 0.376953125, 0.049072265625, 0.3...","[0.08544921875, 0.22265625, 0.1689453125, 0.06...","[0.29296875, 0.341796875, -0.1484375, 0.107910...",False,function match_parens(lst){\n let count = 0...,0
194,HumanEval_96_count_up_to,HumanEval_96_count_up_to_4,269,Here is the completed function:\n\n```cpp\nstd...,"[-0.2275390625, 0.333984375, -0.390625, 0.1552...","[0.23828125, 0.34765625, 0.08642578125, 0.3027...","[0.1484375, 0.412109375, 0.1025390625, 0.14062...","[0.392578125, 0.189453125, -0.271484375, 0.011...",False,std::vector<long> count_up_to(long n) {\n s...,1
13,HumanEval_89_encrypt,HumanEval_89_encrypt_3,232,Here is the completed function:\n\n```csharp\n...,"[-0.2138671875, 0.326171875, -0.375, 0.1328125...","[0.23828125, 0.380859375, 0.095703125, 0.32031...","[0.142578125, 0.376953125, -0.23046875, -0.057...","[0.35546875, 0.208984375, -0.080078125, 0.1103...",False,"string result = """";\n foreach (char c ...",0
